# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# TODO: da rifare per ogni stock
sheet_names = [
    'Info',
    'Historical',
    'Income Statement',
    'Quarterly Income Statement',
    'Cashflow',
    'Institutional Holders',
    'Mutual Fund Holders',
    'Major Holders'
]

#riempire stocks di tutti i vari codici, fare la retrive di tutti i file e buttarli in df_stock per poi poter lavorare su tutti i dati
directory = "./data"
stocks = [os.path.join(directory, file) for file in os.listdir(directory)]
stocks

['./data/GS.xlsx',
 './data/1398.HK.xlsx',
 './data/BA.xlsx',
 './data/005380.KS.xlsx',
 './data/AIR.PA.xlsx',
 './data/KO.xlsx',
 './data/WMT.xlsx',
 './data/005930.KS.xlsx',
 './data/NOVN.SW.xlsx',
 './data/CVX.xlsx',
 './data/EGHT.xlsx',
 './data/MMM.xlsx',
 './data/VZ.xlsx',
 './data/JPM.xlsx',
 './data/JNJ.xlsx',
 './data/AMZN.xlsx',
 './data/9984.T.xlsx',
 './data/RDS.A.xlsx',
 './data/0700.HK.xlsx',
 './data/BE.xlsx',
 './data/AAPL.xlsx',
 './data/PFE.xlsx',
 './data/SEDG.xlsx',
 './data/MSFT.xlsx',
 './data/TM.xlsx',
 './data/VOW3.DE.xlsx',
 './data/PG.xlsx',
 './data/DB.xlsx',
 './data/UL.xlsx',
 './data/TWTR.xlsx',
 './data/XOM.xlsx',
 './data/T.xlsx']

## Integrazione dei dati finanziari
Colonne aggiunte:
- **Daily_Return**: rendimento giornaliero.
- **Target_1day**: indica se il prezzo di chiusura del giorno successivo sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.
- **Target_5days**: indica se il prezzo di chiusura a 5 giorni nel futuro sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.
- **Target_30days**: indica se il prezzo di chiusura a 30 giorni nel futuro sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.

Integrato i vari sheet "Income Statement", "Quarterly Income Statement" e "Cashflow" in un singolo excel. NB: Dato che questi fogli contengono dati finanziari annuali o trimestrali un approccio comune è portare avanti l'ultimo valore noto per ogni giorno fino a quando non si dispone di un nuovo valore. Per alcunii anni finanziari sarà Nan perché non li abbiamo.

Lista delle azioni alle quali mancano pezzi:
- **1398.HK** manca income_statement.normalized_EBITA

In [26]:
# TODO: da fare per ogni stock
counter = 0
df = pd.DataFrame()
for file in stocks:
    if file.split("/")[2][:-5] != "1398.HK":
        df_stock = pd.ExcelFile(file)
        
        # prevent false postive warnings, reference_ https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
        pd.options.mode.chained_assignment = None # default='warn'
        
        # Loading the 'Historical' data stock
        historical_data = df_stock.parse('Historical')
        
        # Renaming and setting the Date column
        historical_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
        historical_data['Date'] = pd.to_datetime(historical_data['Date'])
        historical_data.set_index('Date', inplace=True)
        # Calculate daily return
        historical_data['Daily_Return'] = historical_data['Close'].pct_change()
        
        # Create target variables for next day, next 5 days and next 30 days
        historical_data['Target_1day'] = (historical_data['Close'].shift(-1) > historical_data['Close']).astype(int)
        historical_data['Target_5days'] = (historical_data['Close'].shift(-5) > historical_data['Close']).astype(int)
        historical_data['Target_30days'] = (historical_data['Close'].shift(-30) > historical_data['Close']).astype(int)
        
        # Drop rows with NaN values (will be present due to the shifting for target creation)
        historical_data = historical_data.dropna()
        
        # Loading the 'Income Statement' data for XOM
        income_statement = df_stock.parse('Income Statement')
        
        # Transposing the data for easier integration
        income_statement = income_statement.set_index('Unnamed: 0').transpose()
        income_statement.index = pd.to_datetime(income_statement.index)
        
        
        # Selecting some of the key financial metrics (you can add or remove based on relevance)
        selected_metrics = [
            'Normalized EBITDA',
            'Total Unusual Items',
            'Total Unusual Items Excluding Goodwill'
        ]
        
        # check if columns exist, in case create them
        for metric in selected_metrics:
            if metric not in income_statement.columns:
                income_statement[metric] = np.nan
                
        
        income_statement = income_statement[selected_metrics]
        
        # Merging the income statement data with the historical data
        merged_data = historical_data.join(income_statement, how='left')
        
        # Forward filling the NaN values
        merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
        
        # Loading the 'Cashflow' data for XOM
        cashflow = df_stock.parse('Cashflow')
        
        # Transposing the data for easier integration
        cashflow = cashflow.set_index('Unnamed: 0').transpose()
        cashflow.index = pd.to_datetime(cashflow.index)
        
        # Selecting some of the key cashflow metrics (you can add or remove based on relevance)
        selected_cashflow_metrics = [
            'Operating Cash Flow',
            'Capital Expenditure',
            'Free Cash Flow'
        ]
        
        for metric in selected_cashflow_metrics:
            if metric not in cashflow.columns:
                cashflow[metric] = np.nan
        
        cashflow = cashflow[selected_cashflow_metrics]
        
        # Merging the cashflow data with the existing dataframe
        merged_data = merged_data.join(cashflow, how='left', rsuffix='_cashflow')
        
        # Forward filling the NaN values
        merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')
        
        if 'Ticker' not in merged_data.columns:
            merged_data['Ticker'] = file.split("/")[2].replace(".", "")[:-4]
        
        # Display the updated dataframe with integrated cashflow metrics
        merged_data.iloc[counter : counter + len(merged_data), merged_data.columns.get_loc("Ticker")] = file.split("/")[2].replace(".", "")[:-4]
        
        counter = len(merged_data)

        # Moving Averages
        merged_data['MA_5'] = merged_data['Close'].rolling(window=5).mean()
        merged_data['MA_10'] = merged_data['Close'].rolling(window=10).mean()
        merged_data['MA_30'] = merged_data['Close'].rolling(window=30).mean()
        merged_data['MA_50'] = merged_data['Close'].rolling(window=50).mean()
        
        # RSI
        delta = merged_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        rs = avg_gain / avg_loss
        merged_data['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        merged_data['MACD'] = merged_data['Close'].ewm(span=12, adjust=False).mean() - merged_data['Close'].ewm(span=26, adjust=False).mean()
        merged_data['Signal_Line'] = merged_data['MACD'].ewm(span=9, adjust=False).mean()
        
        # Bollinger Bands
        merged_data['Bollinger_Mid_Band'] = merged_data['Close'].rolling(window=20).mean()
        merged_data['Bollinger_Upper_Band']  = merged_data['Bollinger_Mid_Band'] + 1.96*merged_data['Close'].rolling(window=20).std()
        merged_data['Bollinger_Lower_Band']  = merged_data['Bollinger_Mid_Band'] - 1.96*merged_data['Close'].rolling(window=20).std()
        
        # Volatility
        merged_data['Volatility'] = merged_data['Daily_Return'].rolling(window=5).std()
        
        to_drop_na = ['MA_5', 'MA_10', 'MA_30', 'MA_50', 'RSI', 'Volatility']
        
        for column in to_drop_na:
            merged_data[column] = merged_data[column].fillna(0)

        merged_data = merged_data[merged_data.index >= '2020-06-30']
        indices_to_drop = merged_data.index[merged_data.isna().sum(axis=1) > 3].tolist()
        
        merged_data.drop(indices_to_drop, inplace=True)
        
        # Export in Excel company data
        if not os.path.exists('./Processed'):
            os.makedirs('./Processed')
        with pd.ExcelWriter(f'./Processed/{file.split("/")[2][:-5]}.xlsx', mode = "w", engine = "openpyxl") as writer:
            merged_data.to_excel(writer, sheet_name="Sheet1")
        # Append to one single dataframe
        df = pd.concat([df, merged_data])

# aggiornamento
dato che alcune aziende hanno delle colonne completamente vuote, eliminare i record con i NaN implcava che l'intera azienda non sarebbe stata considerata.
per questo motivo abbiamo deciso si non rimuovere le aziende, ma bensì non inserire nel dataset quelle colonne; in particola le colonne che devono essere rimosse sono:
- `Normalized EBITDA`
- `Total Unusual Items`
- `Total Unusual Items Excluding Goodwill`
- `Operating Cash Flow`
- `Capital Expenditure`
- `Free Cash Flow`
di tutte queste colonne alcune in realtà possono essere lasciate, bisogna valutare bene quali

In [28]:
df.Ticker.unique()
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Return,Target_1day,Target_5days,...,MA_10,MA_30,MA_50,RSI,MACD,Signal_Line,Bollinger_Mid_Band,Bollinger_Upper_Band,Bollinger_Lower_Band,Volatility
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-30,179.305975,183.533325,179.102469,182.802551,3102800,0.0,0.0,0.021451,0,1,...,185.490639,185.971378,177.444583,40.980261,0.824024,3.006285,190.221655,205.998887,174.444424,0.053545
2020-07-01,183.968046,184.763564,180.859977,182.756271,2620100,0.0,0.0,-0.000253,0,0,...,184.693272,186.614103,177.904128,52.499987,0.573090,2.519646,189.620392,205.574185,173.666599,0.051360
2020-07-02,187.316639,187.779150,182.349284,182.599030,2699400,0.0,0.0,-0.000860,1,1,...,184.097562,187.140968,178.320632,46.428569,0.357415,2.087200,188.814700,204.471989,173.157410,0.044924
2020-07-06,186.243599,192.209977,186.049352,191.812225,3567700,0.0,0.0,0.050456,0,1,...,184.627597,188.016001,178.938497,50.786539,0.919322,1.853625,188.326289,202.922628,173.729949,0.021055
2020-07-07,190.091720,190.285980,184.254842,184.412094,2853500,0.0,0.0,-0.038580,1,1,...,184.252042,188.649572,179.372509,42.843173,0.758762,1.634652,187.334206,200.017998,174.650413,0.032740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-09,14.470000,14.750000,14.460000,14.730000,30659800,0.0,0.0,0.019377,1,0,...,14.580206,14.557477,14.300160,45.736901,0.044443,0.082328,14.684344,15.194776,14.173913,0.012248
2023-10-10,14.690000,14.980000,14.690000,14.960000,35746800,0.0,0.0,0.015614,0,0,...,14.602363,14.591356,14.314404,43.159902,0.070110,0.079884,14.722898,15.192227,14.253568,0.013216
2023-10-11,14.950000,15.050000,14.680000,14.770000,31793500,0.0,0.0,-0.012701,0,0,...,14.620238,14.600586,14.329165,44.079070,0.074263,0.078760,14.744101,15.180185,14.308016,0.014393


In [8]:
# TODO: spiegare perchè togliamo i quarterly
#merged_data.drop(columns=['Normalized EBITDA_quarterly', 'Total Unusual Items_quarterly', 'Total Unusual Items Excluding Goodwill_quarterly'], inplace=True)
merged_data.iloc[1 : counter + len(merged_data), merged_data.columns.get_loc("Ticker")] = file.split("/")[2][:-5]
merged_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Return,Target_1day,Target_5days,Target_30days,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Operating Cash Flow,Capital Expenditure,Free Cash Flow,Ticker
Date,,,,,,,,,,,,,,,,,,
1983-11-22,0.00,0.288369,0.286029,0.286029,3884616,0.0,0.0,-0.008114,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,T
1983-11-23,0.00,0.286614,0.283690,0.284859,2143291,0.0,0.0,-0.004090,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,T
1983-11-25,0.00,0.286614,0.284859,0.285444,4475650,0.0,0.0,0.002053,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,T
1983-11-28,0.00,0.286029,0.282520,0.282520,5713325,0.0,0.0,-0.010246,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,T
1983-11-29,0.00,0.281935,0.277840,0.277840,3039374,0.0,0.0,-0.016564,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-09,14.47,14.750000,14.460000,14.730000,30659800,0.0,0.0,0.019377,1,0,0,6.159700e+10,-4.904000e+09,-4.904000e+09,4.195700e+10,-1.652700e+10,2.543000e+10,T
2023-10-10,14.69,14.980000,14.690000,14.960000,35746800,0.0,0.0,0.015614,0,0,0,6.159700e+10,-4.904000e+09,-4.904000e+09,4.195700e+10,-1.652700e+10,2.543000e+10,T
2023-10-11,14.95,15.050000,14.680000,14.770000,31793500,0.0,0.0,-0.012701,0,0,0,6.159700e+10,-4.904000e+09,-4.904000e+09,4.195700e+10,-1.652700e+10,2.543000e+10,T


## Feature Engineering
- **Medie mobili**: Calcoliamo le medie mobili a breve e lungo termine per il prezzo di chiusura, che sono comuni nel trading algoritmico. Ad esempio, medie mobili a 5, 10, 30 e 50 giorni.
- **RSI (Relative Strength Index)**: Questo è un indicatore di momentum che può aiutare a identificare se un'azione è in condizione di "overbought" o "oversold".
- **MACD (Moving Average Convergence Divergence)**: Un altro indicatore di momentum.
- **Bollinger Bands**: Questi sono basati su medie mobili e possono aiutare a identificare se un prezzo è relativamente alto o basso.
- **Volatilità**: Potremmo calcolare la volatilità come la deviazione standard dei rendimenti giornalieri in una finestra temporale specifica.

In [ ]:
# TODO: da fare per ogni stock
# TODO: controllare gpt

# Moving Averages
merged_data['MA_5'] = merged_data['Close'].rolling(window=5).mean()
merged_data['MA_10'] = merged_data['Close'].rolling(window=10).mean()
merged_data['MA_30'] = merged_data['Close'].rolling(window=30).mean()
merged_data['MA_50'] = merged_data['Close'].rolling(window=50).mean()

# RSI
delta = merged_data['Close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
merged_data['RSI'] = 100 - (100 / (1 + rs))

# MACD
merged_data['MACD'] = merged_data['Close'].ewm(span=12, adjust=False).mean() - merged_data['Close'].ewm(span=26, adjust=False).mean()
merged_data['Signal_Line'] = merged_data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
merged_data['Bollinger_Mid_Band'] = merged_data['Close'].rolling(window=20).mean()
merged_data['Bollinger_Upper_Band']  = merged_data['Bollinger_Mid_Band'] + 1.96*merged_data['Close'].rolling(window=20).std()
merged_data['Bollinger_Lower_Band']  = merged_data['Bollinger_Mid_Band'] - 1.96*merged_data['Close'].rolling(window=20).std()

# Volatility
merged_data['Volatility'] = merged_data['Daily_Return'].rolling(window=5).std()

to_drop_na = ['MA_5', 'MA_10', 'MA_30', 'MA_50', 'RSI', 'Volatility']

for column in to_drop_na:
    merged_data[column] = merged_data[column].fillna(0)

# Display the dataset with new features
merged_data

In [ ]:
# TODO: spiegare perche tagliamo il numero di record
merged_data = merged_data[merged_data.index >= '2020-06-30']
merged_data

In [ ]:
# da fare per ogni stock
output_filepath = "processed_nomedellostock.xlsx"
len(merged_data)
merged_data.to_excel(output_filepath)


In [ ]:
# TODO list updated at 2023-10-23
# TODO: esportare in Excel il dataset finale
# TODO (opzionale) : refactor cartella Processed dentro a data 
# TODO: tagliare i dati al 30/06/2020

if not os.path.exists('./Processed'):
    os.makedirs('./Processed')
with pd.ExcelWriter(f'./Processed/{file.split("/")[2][:-5]}.xlsx', mode = "w", engine = "openpyxl") as writer:
    merged_data.to_excel(writer, sheet_name="Sheet1")
    # Append to one single dataframe
df = pd.concat([df, merged_data])